In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("MultipleDelim").getOrCreate()
sc = spark.sparkContext

In [6]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")

filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"

df=spark.read.text(filepath + "Delim.csv")
#.option('delimiter','~|') can't be more than one character error
#In scala, use the key word mkString after reading the csv like
# val b = a.map(x => x.mkString.split("\\~\\t").mkString("\t"))

df.show(truncate=0)

+---------------------+
|value                |
+---------------------+
|Name~|Age            |
|Azarudeen, Shahul~|25|
|Michel, Clarke~|26   |
|Virat, Kohli~|28     |
|Andrew, Simond~|37   |
|Geogre, Bush~| 59    |
|Flintoff, David~|12  |
|Adam, James~| 20     |
+---------------------+



In [16]:
header = df.first()[0]
header

'Name~|Age'

In [17]:
schema = header.split('~|')
schema

['Name', 'Age']

In [14]:
df_new = df.filter(df['value'] != header).rdd.map(lambda x: x[0].split('~|')).toDF(schema)
df_new.show(truncate=0)

+-----------------+---+
|Name             |Age|
+-----------------+---+
|Azarudeen, Shahul|25 |
|Michel, Clarke   |26 |
|Virat, Kohli     |28 |
|Andrew, Simond   |37 |
|Geogre, Bush     | 59|
|Flintoff, David  |12 |
|Adam, James      | 20|
+-----------------+---+

